This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
exec(open ('../../config.py').read ()) # Load file where API keys are stored
API_KEY = QUANDL_API_KEY

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
from collections import namedtuple

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url= 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json' \
     '?&start_date=2018-08-21&end_date=2018-08-21&api_key={}' \
        .format(API_KEY)
r = requests.get(url)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

# The top level dictionary only has one key ('dataset'), so let's focus on its value, which is another dictionary.
for key, value in r.json()['dataset'].items(): 
    print(key, ":\n", value, "\n")

id :
 10095370 

dataset_code :
 AFX_X 

database_code :
 FSE 

name :
 Carl Zeiss Meditec (AFX_X) 

description :
 Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704 

refreshed_at :
 2018-08-23T22:31:11.714Z 

newest_available_date :
 2018-08-23 

oldest_available_date :
 2000-06-07 

column_names :
 ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'] 

frequency :
 daily 

type :
 Time Series 

premium :
 False 

limit :
 None 

transform :
 None 

column_index :
 None 

start_date :
 2018-08-21 

end_date :
 2018-08-21 

data :
 [['2018-08-21', None, 74.55, 73.0, 73.9, None, 75511.0, 5575558.0, None, None, None]] 

collapse :
 None 

order :
 None 

database_id :
 6129 



##  These are your tasks for this mini project:

**1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).**

In [5]:
url= 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-12-31&api_key={}'.format(API_KEY)
r = requests.get(url)

**2. Convert the returned JSON object into a Python dictionary.**

In [6]:
r_json = r.json()

**3. Calculate what the highest and lowest opening prices were for the stock in this period.**

The challenge is that the data of interest are contained in a list of lists.  To make accessing the relevant data easier, I start by converting this object into a named tuple. 

(Of course, it would be easiest to use a pd.Dataframe, but the instructions state that only tools from the standard library should be used.)

In [7]:
# Replace spaces in column names and make them lowercase
names = [name.replace(' ', '_').lower() for name in r_json['dataset']['column_names']]

# Create namedtuple class enter an empty list where to store instances
Data = namedtuple('data', names)
data =[]

# Create namedtuple out of 'dataset' (which currently is a list of list) and append to list.
for observation in r_json['dataset']['data']:
    data.append(Data._make(observation))
    
# Sort the data by ascending date
data = sorted(data, key=lambda observation: observation.date)

In [8]:
data[0]

data(date='2017-01-02', open=34.99, high=35.94, low=34.99, close=35.8, change=None, traded_volume=44700.0, turnover=1590561.0, last_price_of_the_day=None, daily_traded_units=None, daily_turnover=None)

In [9]:
# Create another nametuple class to store highest and lowest opening prices, 
# as well as the respective dates
Extrema = namedtuple('Extrema', ['highest_open', 'highest_date', 
                                 'lowest_open', 'lowest_date'])

# Instantiate it and fill it with the first observation
extrema = Extrema._make([data[0].open, data[0].date, data[0].open, data[0].date])

# Loop over the remaining observations to find actual opening highs and lows
for observation in data[1:]:
    try:
        if observation.open > extrema.highest_open:
            extrema = Extrema._make([observation.open, observation.date, 
                                     extrema.lowest_open, extrema.lowest_date])
        elif observation.open < extrema.lowest_open:
            extrema = Extrema._make([extrema.highest_open, extrema.highest_date, 
                                     observation.open, observation.date])
    
    # If the opening value is missing, continue to the next observation
    except TypeError:
        continue
    
print('The highest opening price was {high} and occurred on {highest_date}.\n'
      'The lowest opening price was {low} and occurred on {lowest_date}.'
          .format(high=extrema.highest_open, highest_date=extrema.highest_date,
                  low=extrema.lowest_open, lowest_date=extrema.lowest_date))

The highest opening price was 53.11 and occurred on 2017-12-14.
The lowest opening price was 34.0 and occurred on 2017-01-24.


**4. What was the largest change in any one day (based on High and Low price)?**

In [10]:
# Create a nametuple to store the maximum and associated date
Maximum_change_within = namedtuple('Maximum_change_within', ['change', 'date']) 

# Instantiate it by filling it with the value from the first observation:
maximum_change_within = Maximum_change_within._make([data[0].high - data[0].low, 
                                                     data[0].date])

#Loop over the remaining observations to find actual maximum
for observation in data[1:]:
    change = observation.high - observation.low
    if change > maximum_change_within.change:
        maximum_change_within = Maximum_change_within._make([change, observation.date])

print('The largest change in any single day was {} and occurred on {}.'
         .format(maximum_change_within.change, maximum_change_within.date))

The largest change in any single day was 2.8100000000000023 and occurred on 2017-05-11.


**5. What was the largest change between any two days (based on Closing Price)?**

Since we sorted the data by ascending date , we can simply compute the difference between adjacent observations:

In [11]:
# Again create a name to a tuple 1st in which to store the values
Maximum_change_between = namedtuple('Maximum_change_between',
                                    ['change','date_begin', 'date_end'])

# Instantiate it by filling it with the value from the first observation
maximum_change_between = Maximum_change_between._make([data[1].close - data[0].close, 
                                                       data[0].date, 
                                                       data[1].date])

# Loop over the remaining observations
for i in range(1, len(data) - 1):
    change = data[i+1].close - data[i].close
    if change > maximum_change_between.change:
        maximum_change_between = Maximum_change_between._make([change, 
                                                               data[i].date, 
                                                               data[i+1].date])
    
print('The largest change between any two days(based on closing price) '
      'was {} and occurred between {} and {}.'
         .format(maximum_change_between.change, maximum_change_between.date_begin, 
                 maximum_change_between.date_end))

The largest change between any two days(based on closing price) was 1.7199999999999989 and occurred between 2017-05-10 and 2017-05-11.


**6. What was the average daily trading volume during this year?**

In [12]:
# Store the trading volume for each day in a list
daily_trading_volume =[]
for observation in data:
    daily_trading_volume.append(observation.traded_volume)
    
# Compute mean
mean_trading_volume = sum(daily_trading_volume)/len(daily_trading_volume)

print('The average daily trading volume during the year was {}.'
          .format(mean_trading_volume))

The average daily trading volume during the year was 89124.33725490196.


**7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)**

In [13]:
# First check whether we have an even or odd number of observations
len(daily_trading_volume)

255

In [15]:
# Since it is odd, we can simply take the middle value
# (Round down to account for zero-based indexing)
middle_position = int(len(daily_trading_volume)/2 - .5) 
median_training_volume = sorted(daily_trading_volume)[middle_position]

print('The median trading volume during the year was {}.'
          .format(median_training_volume))

The median trading volume during the year was 76286.0.
